In [9]:
from pynvml import smi

In [10]:
nvsmi = smi.nvidia_smi.getInstance()

In [11]:
nvsmi

In [12]:
result = nvsmi.DeviceQuery('memory.free, memory.total')

In [13]:
type(result)


dict

In [15]:
nvsmi.DeviceQuery([smi.NVSMI_UTILIZATION_GPU, smi.NVSMI_UTILIZATION_MEM] )

{'gpu': [{'utilization': {'gpu_util': 0, 'memory_util': 0, 'unit': '%'}},
  {'utilization': {'gpu_util': 0, 'memory_util': 0, 'unit': '%'}}]}

In [ ]:
def report_gpu_metric(value, type, instance_id, zone, project_id):
    series = monitoring_v3.types.TimeSeries()
    series.metric.type = 'custom.googleapis.com/{type}'.format(type=type)
    series.resource.type = 'gce_instance'
    series.resource.labels['instance_id'] = instance_id
    series.resource.labels['zone'] = zone
    series.resource.labels['project_id'] = project_id
    point = series.points.add()
    point.value.int64_value = value
    now = time.time()
    point.interval.end_time.seconds = int(now)
    point.interval.end_time.nanos = int(
        (now - point.interval.end_time.seconds) * 10**9)
    client.create_time_series(project_name, [series])

In [16]:
from google.cloud import monitoring_v3
from google.cloud.monitoring_v3.query import Query
import datetime

In [65]:
client = monitoring_v3.MetricServiceClient()
project_name = f'projects/jk-mlops-dev'
filter = 'metric.type = starts_with("custom.googleapis.com/opencensus")'


In [86]:
for descriptor in client.list_metric_descriptors(name=project_name, filter_=filter):
    print(f'Descriptor: {descriptor.name}')
    print(descriptor)

In [85]:
for descriptor in client.list_metric_descriptors(name=project_name, filter_=filter):
    print(f'Deleting: {descriptor.name}')
    client.delete_metric_descriptor(name=descriptor.name)

In [87]:
project_id = 'jk-mlops-dev'
metric_type = 'custom.googleapis.com/opencensus/gce/gpu/utilization_distribution'

In [88]:
START_TIME = datetime.datetime.fromisoformat('2021-01-22T04:00:00-00:00')
END_TIME = datetime.datetime.fromisoformat('2021-01-22T05:00:00-00:00')

In [91]:
query = Query(client, project_id, metric_type=metric_type)
query = query.select_interval(END_TIME, START_TIME)
#query = query.select_resources(instance_id='1297153725150009955')
#query = query.select_metrics(opencensus_task='py-13989@jk-test')

In [94]:
df = query.as_dataframe()
df

resource_type                                                    gce_instance  \
project_id                                                       jk-mlops-dev   
zone                                                               us-west1-b   
instance_id                                               5382047594078986292   
device                                                                      0   
2021-01-22 04:02:11.137948  count: 6\nbucket_options {\n  explicit_buckets...   
2021-01-22 04:02:41.138088  count: 12\nbucket_options {\n  explicit_bucket...   
2021-01-22 04:03:11.138221  count: 18\nbucket_options {\n  explicit_bucket...   
2021-01-22 04:03:41.138345  count: 24\nbucket_options {\n  explicit_bucket...   
2021-01-22 04:04:11.138576  count: 30\nbucket_options {\n  explicit_bucket...   
2021-01-22 04:04:41.138740  count: 36\nbucket_options {\n  explicit_bucket...   
2021-01-22 04:05:11.138878  count: 42\nbucket_options {\n  explicit_bucket...   

resource_type                                                                  
project_id                                                                     
zone                                                                           
instance_id                                                                    
device                                                                      1  
2021-01-22 04:02:11.137948  count: 6\nbucket_options {\n  explicit_buckets...  
2021-01-22 04:02:41.138088  count: 12\nbucket_options {\n  explicit_bucket...  
2021-01-22 04:03:11.138221  count: 18\nbucket_options {\n  explicit_bucket...  
2021-01-22 04:03:41.138345  count: 24\nbucket_options {\n  explicit_bucket...  
2021-01-22 04:04:11.138576  count: 30\nbucket_options {\n  explicit_bucket...  
2021-01-22 04:04:41.138740  count: 36\nbucket_options {\n  explicit_bucket...  
2021-01-22 04:05:11.138878  count: 42\nbucket_options {\n  explicit_bucket...